<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/git_teste_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa pacotes que serão usados

In [13]:
#!rm -rf cassandra*

In [40]:
import pandas as pd
import os

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try: 
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')
  
def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit 
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd cassandra && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit 
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try: 
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta cassandra
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio cassandra
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta cassandra
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto cassandra)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto cassandra
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto cassandra
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do cassandra
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1]) 
      lista_comment.append(each[2]) 
      lista_code.append(each[3])  
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp, 
    'files':lista_files, 
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))	
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))	
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [15]:
instala_cloc()

['', 'Reading package lists... 0%', '', 'Reading package lists... 0%', '', 'Reading package lists... 0%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 45%', '', 'Reading package lists... 45%', '', 'Reading package lists... 46%', '', 'Reading package lists... 46%', '', 'Reading package lists... 54%', '', 'Reading package lists... 56%', '', 'Reading package lists... 56%', '', 'Reading package lists... 64%', '', 'Reading package lists... 64%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 79%', '', 'Reading package lists... 79%', '', 'Reading package lists... 86%', '', 'Reading package lists... 86%'

# Clona repositorio principal

In [16]:
clona_repositorio(url='https://github.com/apache/cassandra.git')
print(f"Primeiro commit do cassandra: {get_primeiro_commit('cassandra')}")
print(f"Ultimo commit do cassandra: {get_ultimo_commit('cassandra')}")

Clonando o repositório https://github.com/apache/cassandra.git
git clone https://github.com/apache/cassandra.git
concluido com sucesso!
Primeiro commit do cassandra: ['1f91e99223b0d1b7ed8390400d4a06ac08e4aa85']
Ultimo commit do cassandra: ['4e3447ae1727a0d01c9d81792acbdeeef94d86d2']


# Cria um repositorio apontando para o 1o. commit

In [6]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_c1')
# git checkout $( git rev-list --all | tail -n 1 )
#!cd $REPOSITORIO_INICIAL && git checkout 1f91e99223b0d1b7ed8390400d4a06ac08e4aa85
primeiro_commit = get_primeiro_commit(repositorio='cassandra')
checkout_n_commit('cassandra_c1', primeiro_commit[0])
df_info_repo = gera_df_info_repositorio(pasta_n_commit='cassandra_c1')
df_dados_linguagens = gera_df_dados(pasta_n_commit='cassandra_c1')

Checkout do repositorio no commit 1f91e99223b0d1b7ed8390400d4a06ac08e4aa85 realizado com sucesso!


# Cria um repositório apontando para o último commit

In [48]:
# cria pasta o primeiro/ultimo commit
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_cf')
# pega o primero/ultimo commit
ultimo_commit = get_ultimo_commit(repositorio='cassandra')
# faz o checkout para primeiro/ultimo commit
checkout_n_commit('cassandra_cf', ultimo_commit[0])
# gera dataframe de info do repositorio
df_info_repo_cf = gera_df_info_repositorio(pasta_n_commit='cassandra_cf')
# gera dataframe de dados linguagem do repositorio
df_dados_linguagens_cf = gera_df_dados(pasta_n_commit='cassandra_cf')

Checkout do repositorio no commit 4e3447ae1727a0d01c9d81792acbdeeef94d86d2 realizado com sucesso!


# Gera dataframe com os dados dos repositorios

In [49]:
df_repo = gera_df_dados_repositorios(df_info_repo, df_info_repo_cf)
df_repo

,descricao,dados_c1,dados_cf
0,nome,cassandra-3.11.11,cassandra_cf
1,qtd_arquivos,8791,5186
2,qtd_commits,24000,28432
3,tamanho_pasta_mb,979,486
4,qtd_colaboradores_commits,405,578
5,data_primeiro_commit,2009-03-02,2009-03-02
6,data_ultimo_commit,2021-07-25,2023-03-10


In [50]:
df_dados_linguagens_cf

,linguagens,files,blank,comment,code
0,[Java],3877,116507,146654,613993
1,[YAML],45,767,2563,330789
2,[AsciiDoc],109,5592,11,20115
3,[Python],31,2460,3163,8332
4,[XML],33,201,478,3740
5,[Markdown],13,773,0,3387
6,[SVG],17,197,1399,2937
7,"[Bourne, Shell]",82,369,987,1970
8,"[ANTLR, Grammar]",3,267,288,1897
9,[Ant],3,185,233,1787


In [52]:
df_repo_linguagens = gera_df_dados_repositorios(df_dados_linguagens_cf, df_dados_linguagens_cf)
df_repo_linguagens

,descricao,dados_c1,dados_cf
0,linguagens,[Java],[Java]
1,files,3877,3877
2,blank,116507,116507
3,comment,146654,146654
4,code,613993,613993


In [ ]:
# Lista as tags registradas do repositorio cassandra
lista_tags = !cd cassandra && git tag

In [11]:
# Pega o primeiro commit da tag alvo
!cd cassandra && git rev-list -n 1 cassandra-3.0.0

96f407bce56b98cd824d18e32ee012dbb99a0286


# Testes com as tags do cassandra

In [44]:
# Cria listas para armazenar os dataframes de cada tag
lista_df_info_repo = []
lista_df_dados_linguagens =[]
lista_df_repo = []
lista_df_repo_linguagens = []

# Lista de tags das analises do repositorio do cassandra
lista_my_tags = ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2',
 'cassandra-3.3',
 'cassandra-3.4',
 'cassandra-3.5',
 'cassandra-3.6',
 'cassandra-3.7',
 'cassandra-3.8',
 'cassandra-3.9',
 'cassandra-3.10',
 'cassandra-3.11.0',
 'cassandra-3.11.11']

for my_tag in lista_my_tags: 
  print(f'Analisando a tag: {my_tag}')
  # 1. Cria uma pasta com o nome da tag alvo
  cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit=my_tag)
  # 2. Pega o primeiro_commit da tag alvo
  primeiro_commit = get_commit_tag(tag=my_tag)
  # 3. Faz o checkout do repositorio para a tag alvo
  checkout_n_commit(my_tag, primeiro_commit[0])
  # 4. Gera os dataframes de informacoes do checkout e linguagens do checkout
  df_info_repo = gera_df_info_repositorio(pasta_n_commit=my_tag)
  df_dados_linguagens = gera_df_dados(pasta_n_commit=my_tag)
  print(f'Análise da tag: {my_tag} concluída com sucesso!')

  print(f'Gera o dataframe de dados do repo {my_tag}')
  df_repo = gera_df_dados_repositorios2(df_info_repo, df_info_repo)
  lista_df_repo.append(df_repo)

  print(f'Gera o dataframe de dados das linguagens do repo {my_tag}')
  df_repo_linguagens_final = gera_df_dados_repositorios2(df_dados_linguagens, df_dados_linguagens)
  lista_df_repo_linguagens.append(df_repo_linguagens_final)

  # 5. Registra a tupla (my_tag, df) das informacoes e linguagens do checkout
  lista_df_info_repo.append((my_tag,df_info_repo))
  lista_df_dados_linguagens.append((my_tag,df_dados_linguagens))

Analisando a tag: cassandra-3.0.0
Checkout do repositorio no commit 96f407bce56b98cd824d18e32ee012dbb99a0286 realizado com sucesso!
Análise da tag: cassandra-3.0.0 concluída com sucesso!
Gera o dataframe de dados do repo cassandra-3.0.0
Gera o dataframe de dados das linguagens do repo cassandra-3.0.0
Analisando a tag: cassandra-3.1
Checkout do repositorio no commit e092873728dc88aebc6ee10153b9bd3cd90cd858 realizado com sucesso!
Análise da tag: cassandra-3.1 concluída com sucesso!
Gera o dataframe de dados do repo cassandra-3.1
Gera o dataframe de dados das linguagens do repo cassandra-3.1
Analisando a tag: cassandra-3.2
Checkout do repositorio no commit 3c6dfa4aa0b9ffb0a48a02b949bff2a8406764e6 realizado com sucesso!
Análise da tag: cassandra-3.2 concluída com sucesso!
Gera o dataframe de dados do repo cassandra-3.2
Gera o dataframe de dados das linguagens do repo cassandra-3.2
Analisando a tag: cassandra-3.3
Checkout do repositorio no commit 5669c6967bbdd540f27aeebf5a2c258bc4defbe3 rea

In [45]:
# Faz um merge dos dataframes gerados
# df_final = df1 + df2 + ... + dfN

df_dados_final = pd.concat(lista_df_repo, axis=1)
df_linguagens_final = pd.concat(lista_df_repo_linguagens, axis=1)

In [46]:
df_dados_final

,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf
0,cassandra-3.0.0,cassandra-3.1,cassandra-3.2,cassandra-3.3,cassandra-3.4,cassandra-3.5,cassandra-3.6,cassandra-3.7,cassandra-3.8,cassandra-3.9,cassandra-3.10,cassandra-3.11.0,cassandra-3.11.11
1,7592,7605,7653,7662,7830,7968,8014,8082,8249,8249,8383,8419,8791
2,18441,18760,19538,19763,20083,20266,20602,20769,21345,21347,22057,22438,24000
3,983,983,984,984,989,992,994,995,997,997,1000,1001,979
4,201,208,220,225,234,238,250,257,279,279,304,321,405
5,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02,2009-03-02
6,2015-11-06,2015-12-04,2016-01-07,2016-02-03,2016-02-16,2016-04-07,2016-05-30,2016-06-06,2016-09-26,2016-09-26,2017-01-30,2017-06-19,2021-07-25


In [47]:
df_linguagens_final

,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf,dados_cf
0,[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java],[Java]
1,5511,5518,5551,5560,5663,5669,5695,5698,5697,5697,5713,5722,5861
2,174145,174589,175543,175954,180596,181204,182756,182911,184201,184201,186561,187310,194015
3,211420,211849,212613,212957,216684,217056,218538,218948,221767,221767,223378,224056,229999
4,924810,927394,933262,935202,955709,958576,967225,968232,974199,974199,986189,989388,1025548
